# Availability Calculator

This tool estimates the average device availability over a period of time.

Double-click into the cells below, where it says `'here'`, and adjust the values as necessary.

After setting configuration values, select `Kernel` > `Restart & Run All` from the menu.

In [ ]:
from datetime import datetime, timedelta
import time
import query
from measure import DeviceCounter
import pandas
from statistics import mean

## `provider_name`

Valid choices are (casing matters):

* `bird`
* `JUMP`
* `Lime`
* `Lyft`

In [ ]:
### Configuration ###

provider_name = 'here'

#####################

print(f"Provider: {provider_name}")

## `vehicle_type`

Valid choices are (casing matters):

* `bicycle` - `JUMP` only
* `scooter` - all providers

In [ ]:
### Configuration ###

vehicle_type = 'here'

#####################

print(f"Vehicle Type: {vehicle_type}")

## `start_date`:

In [ ]:
### Configuration ###

start_year = 2018
start_month = 11
start_day = 0

#####################

start_date = datetime(start_year, start_month, start_day, 0, 0, 0)
print("Starting:", start_date)

## `end_date`:

In [ ]:
### Configuration ###

end_year = 2018
end_month = 11
end_day = 0

#####################

end_date = datetime(end_year, end_month, end_day, 23, 59, 59)
print("Ending:", end_date)

## Query for availability data:

In [ ]:
q = query.Availability(start_date, end_date, vehicle_types=vehicle_type, table="csm_availability_windows", local=True, debug=True)
data = q.get(provider_name=provider_name)

## Count availability in a partitioned time range:

In [ ]:
# create a device counter for the time range, assuming local time
devices = DeviceCounter(start_date, end_date, local=True, debug=True)

# create the interval partition and aggregate counts
partition = devices.count(data).partition()

In [ ]:
partition.describe()

## Average availability:

Over the computed interval partition.

In [ ]:
overall_avg = devices.average()
print(f"Overall average: {overall_avg}")

## Count availability (again), day-by-day:

Calculate average availability for each day in the range `start_date` to `end_date`.

At the end, calculate the overall average.

In [ ]:
oneday = timedelta(days=1)
counts = {}
start = start_date

while start < end_date:
    end = start + oneday
    print(f"Counting {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")
    
    q = query.Availability(start, end, vehicle_types=vehicle_type, table="csm_availability_windows", local=True, debug=False)
    data = q.get(provider_name=provider_name)
    print(f"{len(data)} availability records in time period")
    
    counter = DeviceCounter(start, start + oneday, local=True, debug=False)
    counts[start] = counter.count(data)
   
    start = start + oneday
    print()

print("Done counting. Daily averages:")
print()

for date, count in counts.items():
    print(f"{provider_name},{vehicle_type},{date.strftime('%Y-%m-%d')},{count.average()},{overall_avg}")